# Template - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

/home/zuongthao/PycharmProjects/algo-stock/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

remote


In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-13 14:15:00,1333.5,1335.4,1333.4,1335.0,5583
2025-02-13 14:20:00,1335.4,1336.5,1334.7,1334.9,5753
2025-02-13 14:25:00,1335.3,1336.8,1334.9,1336.8,4879


In [6]:
def get_1st_condition(r):
    signal = ''
    if r['Close'] > r['Close_s1'] > r['Close_s2']:
        # close price increasing
        signal = 'long'
    elif r['Close'] < r['Close_s1'] < r['Close_s2']:
        # close price decreasing
        signal = 'short'
    return signal

def get_2nd_condition(r):
    signal = ''
    if r['open_close'] > 0 and r['open_close_s1'] > 0 :
        # 2 last candlesticks are green
        signal = 'long'
    elif r['open_close'] < 0 and r['open_close_s1'] < 0:
        # 2 last candlesticks are red
        signal = 'short'
    return signal

def get_3rd_condition(r):
    reward_on_risk = 3
    signal = ''
    if r['open_close'] > 0:
        risk = r['Close'] - r['min_5']
        posible_reward = r['max_12'] - r['Close']
        if posible_reward > reward_on_risk * risk:
            signal = 'long'
    elif r['open_close'] < 0:
        risk = r['max_5'] - r['Close']
        posible_reward = r['Close'] - r['min_12']
        if posible_reward > reward_on_risk * risk:
            signal = 'short'
    return signal

    
def cal_signal(r):
    signal = ''
    if r['condition_1'] == 'long' and r['condition_2'] == 'long' and r['condition_3'] == 'long':
        signal = 'long'
    elif r['condition_1'] == 'short' and r['condition_2'] == 'short' and r['condition_3'] == 'short':
        signal = 'short'
    return signal

def prepare_data(data):
    data['Close_s1'] = data['Close'].shift(1)
    data['Close_s2'] = data['Close'].shift(2)
    data['Close_s3'] = data['Close'].shift(3)
    data['open_close'] = data['Close'] - data['Open']
    data['open_close_s1'] = data['open_close'].shift(1)
    data['open_close_s2'] = data['open_close'].shift(2)
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['max_12'] = data['High'].rolling(12).max()
    data['min_12'] = data['Low'].rolling(12).min()
    data['condition_1'] = data.apply(lambda r: get_1st_condition(r), axis=1)
    data['condition_2'] = data.apply(lambda r: get_2nd_condition(r), axis=1)
    data['condition_3'] = data.apply(lambda r: get_3rd_condition(r), axis=1)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    return data

In [7]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [8]:
prepared_data[prepared_data.signal != '']

,Open,High,Low,Close,Volume,Close_s1,Close_s2,Close_s3,open_close,open_close_s1,open_close_s2,max_5,min_5,max_12,min_12,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,,,,,,
2020-11-02 14:00:00,901.8,902.4,901.2,901.5,3062,902.0,902.5,900.4,-0.3,-0.5,2.1,903.0,897.0,903.0,896.3,short,short,short,short
2020-11-03 09:15:00,908.6,908.9,908.3,908.5,1355,908.6,908.8,908.8,-0.1,-0.2,0.2,909.4,904.0,909.4,896.6,short,short,short,short
2020-11-03 09:20:00,908.5,908.8,908.1,908.1,847,908.5,908.6,908.8,-0.4,-0.1,-0.2,909.4,908.1,909.4,896.6,short,short,short,short
2020-11-03 09:25:00,908.2,908.6,907.7,907.8,1999,908.1,908.5,908.6,-0.4,-0.4,-0.1,908.9,907.7,909.4,896.6,short,short,short,short
2020-11-03 09:45:00,907.8,908.1,907.4,907.7,1932,907.9,908.8,908.0,-0.1,-0.7,0.8,908.8,907.4,909.4,904.0,short,short,short,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-10 09:50:00,1327.5,1328.0,1326.8,1327.9,2759,1327.5,1327.0,1332.0,0.4,0.3,-4.9,1333.6,1325.2,1343.3,1325.2,long,long,long,long
2025-02-10 10:45:00,1332.6,1333.0,1331.7,1332.1,1450,1332.6,1332.8,1331.7,-0.5,-0.2,1.1,1333.8,1331.2,1335.0,1326.8,short,short,short,short
2025-02-12 09:20:00,1340.4,1340.5,1339.3,1339.3,2902,1340.3,1340.8,1340.2,-1.1,-0.4,0.6,1341.4,1337.4,1341.4,1328.9,short,short,short,short


In [9]:
prepared_data[(prepared_data.condition_1 != '') & (prepared_data.condition_2 != '')]

,Open,High,Low,Close,Volume,Close_s1,Close_s2,Close_s3,open_close,open_close_s1,open_close_s2,max_5,min_5,max_12,min_12,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,,,,,,
2020-11-02 09:55:00,898.7,901.4,898.6,901.3,3705,898.6,898.1,898.8,2.6,0.5,-0.7,901.4,897.7,901.4,896.5,long,long,,
2020-11-02 10:05:00,900.0,900.4,899.5,899.5,1044,900.5,901.3,898.6,-0.5,-0.8,2.6,901.4,897.7,901.4,896.5,short,short,,
2020-11-02 11:00:00,897.3,897.4,895.4,897.0,3418,897.3,898.3,897.9,-0.3,-0.7,0.5,900.2,895.4,901.5,895.4,short,short,,
2020-11-02 13:05:00,898.6,899.7,898.0,899.4,1642,898.5,898.0,898.4,0.8,0.5,-0.3,899.7,897.1,900.2,895.4,long,long,,
2020-11-02 13:35:00,898.3,899.5,898.0,899.1,3239,898.3,896.7,897.4,0.8,1.7,-0.7,899.5,896.3,900.4,895.5,long,long,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13 13:30:00,1336.5,1338.2,1336.4,1338.0,6901,1336.4,1335.6,1335.9,1.5,0.8,-0.3,1338.2,1333.1,1338.2,1332.5,long,long,,
2025-02-13 14:00:00,1336.1,1336.7,1335.0,1335.7,3366,1336.1,1337.3,1336.9,-0.4,-1.3,0.4,1338.5,1335.0,1338.5,1333.0,short,short,,
2025-02-13 14:05:00,1335.6,1335.8,1334.4,1335.2,4360,1335.7,1336.1,1337.3,-0.4,-0.4,-1.3,1338.5,1334.4,1338.5,1333.1,short,short,,


In [10]:
class MainStrategy(Strategy):
    reward_on_risk = 3
    def init(self):
        self._broker._cash = 1500
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if self.position:
            return 
        signal = self.data.signal[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        min_5 = self.data.min_5[-1]
        max_5 = self.data.max_5[-1]
        if signal == 'long':
            buy_price = close_price
            sl = min_5
            tp = buy_price + self.reward_on_risk * (buy_price - min_5)
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = max_5
            tp = sell_price - self.reward_on_risk * (max_5 - sell_price)
            self.sell(size=1, sl=sl, tp=tp)

In [11]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [12]:
stats

Start                     2020-11-02 09:55:00
End                       2025-02-13 14:45:00
Duration                   1564 days 04:50:00
Exposure Time [%]                    15.38405
Equity Final [$]                    1425.3258
Equity Peak [$]                    1537.31676
Return [%]                           -4.97828
Buy & Hold Return [%]                48.39676
Return (Ann.) [%]                   -1.263288
Volatility (Ann.) [%]                4.379132
Sharpe Ratio                        -0.288479
Sortino Ratio                       -0.474606
Calmar Ratio                        -0.143332
Max. Drawdown [%]                   -8.813706
Avg. Drawdown [%]                   -1.308548
Max. Drawdown Duration     1088 days 22:55:00
Avg. Drawdown Duration       91 days 16:09:00
# Trades                                  817
Win Rate [%]                        28.886169
Best Trade [%]                       2.867263
Worst Trade [%]                     -1.203845
Avg. Trade [%]                    

In [13]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,-1,33,34,901.22955,897.0,4.22955,0.004693,2020-11-02 14:05:00,2020-11-02 14:10:00,None,0 days 00:05:00
1,-1,44,58,908.22745,909.4,-1.17255,-0.001291,2020-11-03 09:20:00,2020-11-03 10:30:00,None,0 days 01:10:00
2,-1,109,110,910.42679,911.8,-1.37321,-0.001508,2020-11-04 10:30:00,2020-11-04 10:35:00,None,0 days 00:05:00
3,1,225,229,906.27180,905.0,-1.27180,-0.001403,2020-11-06 13:05:00,2020-11-06 13:25:00,None,0 days 00:20:00
4,-1,251,251,915.22535,916.5,-1.27465,-0.001393,2020-11-09 09:35:00,2020-11-09 09:35:00,None,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
812,1,54135,54140,1340.90215,1342.9,1.99785,0.001490,2025-02-06 10:45:00,2025-02-06 11:10:00,None,0 days 00:25:00
813,1,54227,54250,1328.29837,1336.0,7.70163,0.005798,2025-02-10 09:55:00,2025-02-10 13:15:00,None,0 days 03:20:00
814,-1,54323,54331,1339.19812,1341.4,-2.20188,-0.001644,2025-02-12 09:25:00,2025-02-12 10:05:00,None,0 days 00:40:00
815,1,54388,54390,1327.79822,1330.6,2.80178,0.002110,2025-02-13 10:35:00,2025-02-13 10:45:00,None,0 days 00:10:00


In [14]:
# bt.plot()